In [1]:
from dataclasses import dataclass
import requests
import time
import duckdb
import pandas as pd
from configparser import ConfigParser
import pgeocode

%load_ext sql
%config SqlMagic.displaylimit = 0

There's a new jupysql version available (0.10.11), you're running 0.10.10. To upgrade: pip install jupysql --upgrade
Deploy FastAPI apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [2]:
config = ConfigParser()
config.read("../../.config")

['../../.config']

In [3]:
API_KEY = config['GCP']['API_KEY']
radius = 1500  # in meters
place_type = 'shopping'  # Example place type
address = 'Luxembourg City, Luxembourg'


In [4]:
def format_coordinates(latitude, longitude):
    return f"{latitude},{longitude}"

def get_coordinates(api_key, address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print(f"Error in response: {data['status']}")
            return None, None
    else:
        print(f"HTTP error: {response.status_code}")
        return None, None


latitude, longitude = get_coordinates(API_KEY, address)

if latitude and longitude:
    formatted_location = format_coordinates(latitude, longitude)
    # print(formatted_location)  # Output will be the coordinates of Luxembourg City
else:
    print("Could not retrieve coordinates.")


In [5]:
# url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&key={API_KEY}"
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&type={place_type}&key={API_KEY}"

response = requests.get(url)
places = response.json()

for place in places['results']:
    print(f"Name: {place['name']}")
    print(f"Address: {place.get('vicinity', 'N/A')}")
    print(f"Place ID: {place['place_id']}")
    print('---')


Name: Luxembourg
Address: Luxembourg
Place ID: ChIJVyzznc1IlUcREG0F0dbRAAQ
---
Name: Hôtel Parc Belle-Vue
Address: 5 Avenue Marie-Thérèse, Luxembourg
Place ID: ChIJ3e9sj9VIlUcRUU6bnYZ2tcE
---
Name: Hotel Vauban
Address: 10 Place Guillaume II, Luxembourg
Place ID: ChIJQVrlNdNIlUcRa3v0FrkGQ-g
---
Name: French Café Hotel
Address: 14 Place d'Armes, Luxembourg
Place ID: ChIJz1niVCtPlUcROFfnez0PXz0
---
Name: Hotel Christophe Colomb
Address: 10 Rue d'Anvers, Luxembourg
Place ID: ChIJHeKzA9FIlUcR_UzTEdulOFs
---
Name: Youth Hostel
Address: 2 Rue du Fort Olisy, Luxembourg
Place ID: ChIJS5A_WTJPlUcRtAr15AXUGNs
---
Name: Novotel Luxembourg Centre
Address: 35 Rue du Laboratoire, Luxembourg
Place ID: ChIJl5q7iM5IlUcROv7uDbapuVo
---
Name: Grand Hotel Cravat
Address: 29 Boulevard Franklin Delano Roosevelt, Luxembourg
Place ID: ChIJi-jQXNNIlUcR77bXMeT_HSw
---
Name: Best Western Plus
Address: 3 Avenue Victor Hugo, Luxembourg
Place ID: ChIJld6G1ClPlUcRbXoEwx9_WUs
---
Name: Meliá Luxembourg
Address: 1 Par

In [ ]:
places['results'][0]

In [14]:
place_id = 'ChIJ3e9sj9VIlUcRUU6bnYZ2tcE'  # Example place ID

details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={API_KEY}"

details_response = requests.get(details_url)
place_details = details_response.json()


print(f"Name: {place_details['result']['name']}")
print(f"Address: {place_details['result']['formatted_address']}")
print(f"Phone Number: {place_details['result'].get('formatted_phone_number', 'N/A')}")
print(f"Website: {place_details['result'].get('website', 'N/A')}")


Name: Hôtel Parc Belle-Vue
Address: 5 Av. Marie-Thérèse, 2132 Hollerich Luxembourg
Phone Number: 45 61 41 1
Website: https://www.goereshotels.com/belle-vue-en/


In [17]:
place_details['status']

'OK'

In [7]:
# Connect to DuckDB
# con = duckdb.connect('places.db')
# df = con.execute('SELECT * FROM places').fetchdf()

In [37]:
conn = duckdb.connect('../../data/raw/places.db')
%sql conn --alias duckdb

In [38]:
%%sql
select column_name from  information_schema.columns
where table_name = 'geonames';

Running query in 'duckdb'

column_name
adminCode1
lng
geonameId
toponymName
countryId
fcl
population
countryCode
name
fclName


In [44]:
%%sql
select distinct(countryId) from geonames

Running query in 'duckdb'

countryId
None
2802361
2921044
3017382
2960313


In [10]:
%%sql
show tables;
# show placedetail;

Running query in 'duckdb'

name
placeDetail
places


In [30]:
# con.close()
conn.close()

In [ ]:
def get_cities_towns(api_key):
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address=Luxembourg&key={api_key}'
    response = requests.get(url)
    data = response.json()
    
    if data['status'] == 'OK':
        results = data['results']
        places = []
        for result in results:
            places.append(result['formatted_address'])
        return places
    else:
        print(f"Error: {data['status']}")


cities_towns = get_cities_towns(API_KEY)
print(cities_towns)


In [15]:
base_url = "http://api.geonames.org/searchJSON"
all_data = []
start_row = 0


params = {
    "country": 'LU',
    "maxRows": 1000,
    "username": username
}

response = requests.get(base_url, params=params)
data = response.json()

In [24]:
data['geonames'][0]['fclName']

'city, village,...'

In [53]:
from dataclasses import dataclass, asdict
from typing import Optional

@dataclass
class GeoName:
    adminCode: str
    longitude: float
    geonameId: int
    toponymName: float
    countryId: int
    fcl:str
    population: int
    countryCode: str
    name: str
    fclName: str
    adminCodes: str
    countryName: str
    fcodeName: str
    adminName: str
    latitude: float
    fcode: str

In [50]:
def get_geonames_data(username, country_code, max_rows=1000):
    base_url = "http://api.geonames.org/searchJSON"
    all_data = []
    start_row = 0
    
    while True:
        params = {
            "country": country_code,
            "maxRows": max_rows,
            "username": username,
            "startRow": start_row
        }
        
        response = requests.get(base_url, params=params)
        data = response.json()
        
        if "geonames" in data:
            geonames = data["geonames"]
            all_data.extend([
                GeoName(
                    adminCode=item['adminCode1'] if item.get('adminCode1') else None,
                    longitude=float(item['lng']),
                    geonameId=int(item['geonameId']),
                    toponymName	= item['toponymName'],
                    countryId=int(item['countryId']) if item.get('countryId') else None,
                    fcl=item['fcl'],
                    population=int(item['population']) if item.get('population') else None,
                    countryCode = item['countryCode'] if item.get('countryCode') else None,
                    name = item['name'],
                    fclName = item['fclName'] if item.get('fclName') else None,
                    adminCodes = item['adminCodes1'] if item.get('adminCodes1') else None,
                    countryName = item['countryName'] if item.get('countryName') else None,
                    fcodeName = item['fcodeName'],
                    adminName = item['adminName1'],
                    latitude  = float(item['lat']),
                    fcode = item['fcode']
                ) for item in geonames
            ])
            
            if len(geonames) < max_rows:
                break
        
            start_row += max_rows
            time.sleep(1)
        else:
            print("Error retrieving data:", data.get("status", {}).get("message", "Unknown error"))
            return []

    return all_data

In [51]:
username = config['GEONAME']['username']  # Replace with your GeoNames username
country_code = "LU"  # Luxembourg

df = get_geonames_data(username,country_code)

In [54]:
df_geoname = pd.DataFrame([asdict(gn) for gn in df])

In [65]:
for lat, lon in df_geoname[['latitude','longitude']].iterrows():
    lat , lon = lat, lon

In [66]:
lat

1258

In [57]:
%%sql
select * from df;


Running query in 'duckdb'

InvalidInputException: Invalid Input Error: Python Object "df" of type "list" found on line "/tmp/ipykernel_22492/1846799833.py:1" not suitable for replacement scans.
Make sure that "df" is either a pandas.DataFrame, duckdb.DuckDBPyRelation, pyarrow Table, Dataset, RecordBatchReader, Scanner, or NumPy ndarrays with supported format

In [18]:
%%sql
show geonames

Running query in 'duckdb'

column_name,column_type,null,key,default,extra
adminCode1,VARCHAR,YES,None,None,None
lng,VARCHAR,YES,None,None,None
geonameId,BIGINT,YES,None,None,None
toponymName,VARCHAR,YES,None,None,None
countryId,VARCHAR,YES,None,None,None
fcl,VARCHAR,YES,None,None,None
population,BIGINT,YES,None,None,None
countryCode,VARCHAR,YES,None,None,None
name,VARCHAR,YES,None,None,None
fclName,VARCHAR,YES,None,None,None
